In [ ]:
import requests
import re
import json
import sqlalchemy as db
from datetime import datetime

In [ ]:
url_part_one = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&q=date:"

url_part_tree = "&sort=-date&rows=-1"



In [ ]:
liste_annees = ["2016","2017","2018","2019","2020","2021"]
liste_mois = ["01","02","03","04","05","06","07","08","09","10","11","12"]
liste_part_two_url = []
for annee in liste_annees:
    for mois in liste_mois:
        if mois in ["01","03","05","07","08","10","12"]:
            liste_part_two_url.append(f"[{annee}-{mois}-01T00:00:00Z TO {annee}-{mois}-31T23:59:59Z]")
        elif mois in ["04","06","09","11"]:
             liste_part_two_url.append(f"[{annee}-{mois}-01T00:00:00Z TO {annee}-{mois}-30T23:59:59Z]")
        elif mois == "02":
            if annee in ["2016","2020"]:
                liste_part_two_url.append(f"[{annee}-{mois}-01T00:00:00Z TO {annee}-{mois}-29T23:59:59Z]")
            else:
                liste_part_two_url.append(f"[{annee}-{mois}-01T00:00:00Z TO {annee}-{mois}-28T23:59:59Z]")

In [ ]:
# for url_part_two in liste_part_two_url:
#     name_file = re.findall("\d{4}-\d{2}",url_part_two)[0]
#     with open(f'json/data_sncf_{name_file}.json', 'w') as mon_fichier: 
#         data = requests.get(url_part_one + url_part_two + url_part_tree).json()
#         json.dump(data, mon_fichier)

In [ ]:
engine = db.create_engine("sqlite:///donnees_sncf.db")
connection = engine.connect()

metadata = db.MetaData()
with engine.connect() as con:
    con.execute("DROP TABLE sncf")

sncf_table = db.Table("sncf", metadata,
             db.Column('id',db.Integer(),primary_key=True),
             db.Column('date',db.String()),
             db.Column('heure',db.String()),
             db.Column('categorie_objet',db.String()),
             db.Column('type_objet', db.String()),
             db.Column('gare',db.String()),
             db.Column('uic_gare',db.String()),
             
             )

metadata.create_all(engine)

In [ ]:
liste_annees = ["2016","2017","2018","2019","2020","2021"]
liste_mois = ["01","02","03","04","05","06","07","08","09","10","11","12"]
liste_file = []
for annee in liste_annees:
    for mois in liste_mois:
        liste_file.append(f"{annee}-{mois}")
for endfile in liste_file:
    print(endfile,datetime.now())
    with open(f'json/data_sncf_{endfile}.json') as mon_fichier:
        data = json.load(mon_fichier)
        for objet in data['records']:
            print("un objet",datetime.now())
            date = objet["fields"]["date"].split("T")[0]
            heure = objet["fields"]["date"].split("T")[1].split("+")[0]
            categorie_objet = objet["fields"]["gc_obo_type_c"]
            type_objet = objet["fields"]["gc_obo_nature_c"]
            try:
                gare = objet["fields"]["gc_obo_gare_origine_r_name"]
            except:
                gare="None"
            try:
                uic_gare = objet["fields"]["gc_obo_gare_origine_r_code_uic_c"]
            except:
                uic_gare="None"
            
            
            with engine.connect() as con:
                insertTable = "INSERT INTO sncf (date, heure, categorie_objet, type_objet,gare,uic_gare) VALUES (:date, :heure, :categorie_objet, :type_objet, :gare, :uic_gare)"
                params = {"date":date, "heure":heure, "categorie_objet":categorie_objet, "type_objet":type_objet, "gare":gare,"uic_gare":uic_gare}
                statement = db.text(insertTable)
                con.execute(statement,params)

